In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import argparse
import timeit
import os

class MLP(nn.Module):
    def __init__(self, input_dim=1974, dropout=0.1):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(input_dim, 3000)
        self.dropout1 = nn.Dropout(dropout)
        self.fc1 = nn.Linear(3000, 50)
        self.dropout2 = nn.Dropout(dropout)
        self.output_layer = nn.Linear(50, 1)
        
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = torch.sigmoid(self.output_layer(x))
        return x

def load_dataset(args, device):
    data = np.load(args.datafile)['data']
    print(data.shape)
    data = preprocessing.minmax_scale(data)
    
    np.random.seed(123)
    np.random.shuffle(data)
    
    train, test = train_test_split(data, test_size=0.2, random_state=123)

    train_x, train_y = train[:, :-1], train[:, -1]
    test_x, test_y = test[:, :-1], test[:, -1]

    # create torch tensor from numpy array
    train_x = torch.FloatTensor(train_x).to(device)
    train_y = torch.FloatTensor(train_y).to(device)
    test_x = torch.FloatTensor(test_x).to(device)
    test_y = torch.FloatTensor(test_y).to(device)

    train = torch.utils.data.TensorDataset(train_x, train_y)
    test = torch.utils.data.TensorDataset(test_x, test_y)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=args.batch_size, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=args.batch_size, shuffle=True)

    return train_dataloader, test_dataloader

def train(dataloader, net, optimizer, loss_func, epoch):
    net.train()
    train_loss = 0

    for index, (data, label) in enumerate(dataloader, 1):
        optimizer.zero_grad()
        output = net(data)
        output = torch.flatten(output)
        loss = loss_func(output, label, reduction='mean')
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    print('epoch %4d batch %4d/%4d train_loss %6.3f' % (epoch, index, len(dataloader), train_loss / index), end='')

    return train_loss / index

def test(dataloader, net, loss_func):
    net.eval()
    test_loss = 0

    for index, (data, label) in enumerate(dataloader, 1):
        with torch.no_grad():
            output = net(data)
        output = torch.flatten(output)
        loss = loss_func(output, label, reduction='mean')
        test_loss += loss.item()

    print(' test_loss %6.3f' % (test_loss / index), end='')

    return test_loss / index

def main(args):
    device = torch.device('cuda' if torch.cuda.is_available() and not args.cpu else 'cpu')
    print('Using %s device.' % device)

    train_dataloader, test_dataloader = load_dataset(args, device)

    net = MLP(input_dim=1974, dropout=args.dropout).to(device)

    if args.modelfile:
        net.load_state_dict(torch.load(args.modelfile))

    # define our optimizer and loss function
    optimizer = torch.optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    loss_func = F.binary_cross_entropy

    test_losses = []

    for epoch in range(args.epochs):
        epoch_start = timeit.default_timer()

        train(train_dataloader, net, optimizer, loss_func, epoch)
        test_loss = test(test_dataloader, net, loss_func)

        print(' %5.2f sec' % (timeit.default_timer() - epoch_start))

        test_losses.append(test_loss)

        if test_loss <= min(test_losses):
            torch.save(net.state_dict(), 'model/%5.3f.pth' % min(test_losses))
            
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--datafile', default='data/cpi.npz', type=str)
    parser.add_argument('--modelfile', default=None, type=str)
    parser.add_argument('--epochs', default=500, type=int)
    parser.add_argument('--batch_size', default=100, type=int)
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--weight_decay', default=0., type=float)
    parser.add_argument('--dropout', default=0.1, type=float)
    parser.add_argument('--cpu', action='store_true')
    args = parser.parse_args([])

    os.makedirs('model', exist_ok=True)
    main(args)

Using cpu device.
(20000, 1975)
epoch    0 batch    8/   8 train_loss  0.702 test_loss  0.693  1.34 sec
epoch    1 batch    8/   8 train_loss  0.692 test_loss  0.692  1.30 sec
epoch    2 batch    8/   8 train_loss  0.689 test_loss  0.694  1.27 sec
epoch    3 batch    8/   8 train_loss  0.688 test_loss  0.690  1.23 sec
epoch    4 batch    8/   8 train_loss  0.684 test_loss  0.692  1.19 sec
epoch    5 batch    8/   8 train_loss  0.684 test_loss  0.691  1.16 sec
epoch    6 batch    8/   8 train_loss  0.680 test_loss  0.690  1.26 sec
epoch    7 batch    8/   8 train_loss  0.681 test_loss  0.693  2.10 sec
epoch    8 batch    8/   8 train_loss  0.685 test_loss  0.690  1.54 sec
epoch    9 batch    8/   8 train_loss  0.674 test_loss  0.689  1.48 sec
epoch   10 batch    8/   8 train_loss  0.670 test_loss  0.689  1.17 sec
epoch   11 batch    8/   8 train_loss  0.669 test_loss  0.689  1.23 sec
epoch   12 batch    8/   8 train_loss  0.667 test_loss  0.687  1.24 sec
epoch   13 batch    8/   8 train

KeyboardInterrupt: 